In [224]:
import pandas as pd
import category_encoders as ce
import numpy as np
import joblib

In [225]:
players_21 = pd.read_csv('players_21.csv')
players_22 = pd.read_csv('players_22.csv')

C:\Users\mcnob\AppData\Local\Temp\ipykernel_6412\2679833645.py:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('players_22.csv')


In [226]:
players_21

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,...,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,...,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,132000000.0,270000.0,28,...,49+3,49+3,49+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/21_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,91,129000000.0,370000.0,29,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/21_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,257710,https://sofifa.com/player/257710/mengxuan-zhan...,Zhang Mengxuan,张梦炫,CB,47,52,70000.0,1000.0,21,...,47+2,47+2,47+2,45+2,15+2,https://cdn.sofifa.net/players/257/710/21_120.png,https://cdn.sofifa.net/teams/112165/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
18940,257933,https://sofifa.com/player/257933/wenzhuo-huang...,Huang Wenzhou,黄文卓,CM,47,53,70000.0,1000.0,21,...,46+2,46+2,46+2,47+2,15+2,https://cdn.sofifa.net/players/257/933/21_120.png,https://cdn.sofifa.net/teams/112540/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
18941,257936,https://sofifa.com/player/257936/yue-song/210002,Song Yue,宋岳,CM,47,47,45000.0,2000.0,28,...,46+1,46+1,46+1,47,11+2,https://cdn.sofifa.net/players/257/936/21_120.png,https://cdn.sofifa.net/teams/111774/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
18942,258736,https://sofifa.com/player/258736/vani-da-silva...,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,47,67,130000.0,500.0,17,...,26+2,26+2,26+2,33+2,14+2,https://cdn.sofifa.net/players/258/736/21_120.png,https://cdn.sofifa.net/teams/1920/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/gb-eng.png


In [227]:
players_21.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,...,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,226232.328917,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,...,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,27166.574284,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,...,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,41.000000,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,...,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,210028.750000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,...,50.000000,29.000000,27.000000,24.000000,8.000000,8.000000,8.000000,8.000000,8.000000,28.000000
50%,232301.500000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1913.000000,1.000000,...,59.000000,52.000000,55.000000,52.000000,11.000000,11.000000,11.000000,11.000000,11.000000,37.000000
75%,246745.750000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,110986.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,258970.000000,93.000000,95.000000,1.855000e+08,560000.000000,53.000000,206.000000,110.000000,114899.000000,4.000000,...,96.000000,94.000000,93.000000,90.000000,90.000000,92.000000,93.000000,91.000000,90.000000,65.000000


In [228]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [229]:
object_columns = players_21.select_dtypes(include='object').columns

print(object_columns)

Index(['player_url', 'short_name', 'long_name', 'player_positions', 'dob',
       'club_name', 'league_name', 'club_position', 'club_loaned_from',
       'club_joined', 'nationality_name', 'nation_position', 'preferred_foot',
       'work_rate', 'body_type', 'real_face', 'player_tags', 'player_traits',
       'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
       'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url',
       'club_flag_url', 'nation_logo_url', 'nation_flag_url'],
      dtype='object')


In [230]:
columns_to_drop = players_21.filter(regex='url|dob').columns

players_21.drop(columns=columns_to_drop, axis=1, inplace=True)

In [231]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 103 entries, sofifa_id to gk
dtypes: float64(16), int64(44), object(43)
memory usage: 14.9+ MB


In [232]:
players_21.dropna(thresh= len(players_21) * 0.5, axis=1, inplace=True)

In [233]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  int64  
 1   short_name                   18944 non-null  object 
 2   long_name                    18944 non-null  object 
 3   player_positions             18944 non-null  object 
 4   overall                      18944 non-null  int64  
 5   potential                    18944 non-null  int64  
 6   value_eur                    18707 non-null  float64
 7   wage_eur                     18719 non-null  float64
 8   age                          18944 non-null  int64  
 9   height_cm                    18944 non-null  int64  
 10  weight_kg                    18944 non-null  int64  
 11  club_team_id                 18719 non-null  float64
 12  club_name                    18719 non-null  object 
 13  league_name     

In [234]:
from sklearn.impute import SimpleImputer

numeric_cols = players_21.select_dtypes(include=['number']).columns
num_imputer = SimpleImputer(strategy='median')
players_21[numeric_cols] = num_imputer.fit_transform(players_21[numeric_cols])

obj_imputer = SimpleImputer(strategy='most_frequent')
players_21[players_21.select_dtypes(include='object').columns] = obj_imputer.fit_transform(players_21[players_21.select_dtypes
(include='object').columns])

In [235]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    18944 non-null  float64
 1   short_name                   18944 non-null  object 
 2   long_name                    18944 non-null  object 
 3   player_positions             18944 non-null  object 
 4   overall                      18944 non-null  float64
 5   potential                    18944 non-null  float64
 6   value_eur                    18944 non-null  float64
 7   wage_eur                     18944 non-null  float64
 8   age                          18944 non-null  float64
 9   height_cm                    18944 non-null  float64
 10  weight_kg                    18944 non-null  float64
 11  club_team_id                 18944 non-null  float64
 12  club_name                    18944 non-null  object 
 13  league_name     

In [236]:
columns_to_drop = ['sofifa_id','short_name', 'long_name','player_positions','club_team_id','club_name','league_name',
                   'league_level','club_position','club_jersey_number','club_joined','club_contract_valid_until'
                   ,'nationality_id','nationality_name','real_face','release_clause_eur']
players_21.drop(columns=columns_to_drop, axis=1, inplace=True)

In [237]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 80 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   overall                      18944 non-null  float64
 1   potential                    18944 non-null  float64
 2   value_eur                    18944 non-null  float64
 3   wage_eur                     18944 non-null  float64
 4   age                          18944 non-null  float64
 5   height_cm                    18944 non-null  float64
 6   weight_kg                    18944 non-null  float64
 7   preferred_foot               18944 non-null  object 
 8   weak_foot                    18944 non-null  float64
 9   skill_moves                  18944 non-null  float64
 10  international_reputation     18944 non-null  float64
 11  work_rate                    18944 non-null  object 
 12  body_type                    18944 non-null  object 
 13  pace            

In [238]:
import re

for col in players_21.columns:
    # Convert column to string for checking patterns
    col_str = players_21[col].astype(str)

    # Check if the column contains any data in the form of "number+number" or "number-number"
    if col_str.str.contains(r'^\d+[\+\-]\d+$').any():
        # Apply eval to each element in the column if it matches the pattern
        players_21[col] = players_21[col].apply(lambda x: eval(x) if isinstance(x, str) and re.match(r'^\d+[\+\-]\d+$', x) else x)

        # Convert the column to int or float as needed
        if players_21[col].apply(lambda x: isinstance(x, (int, float))).all():
            players_21[col] = players_21[col].astype(float)

In [239]:
players_21

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,preferred_foot,weak_foot,skill_moves,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,93.0,93.0,103500000.0,560000.0,33.0,170.0,72.0,Left,4.0,4.0,...,68,68,68,69,65,55,55,55,65,22.0
1,92.0,92.0,63000000.0,220000.0,35.0,187.0,83.0,Right,4.0,5.0,...,64,64,64,68,64,57,57,57,64,23.0
2,91.0,91.0,111000000.0,240000.0,31.0,184.0,80.0,Right,4.0,4.0,...,68,68,68,67,64,63,63,63,64,22.0
3,91.0,91.0,132000000.0,270000.0,28.0,175.0,68.0,Right,5.0,5.0,...,65,65,65,70,65,52,52,52,65,23.0
4,91.0,91.0,129000000.0,370000.0,29.0,181.0,70.0,Right,5.0,4.0,...,83,83,83,82,78,72,72,72,78,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,52.0,70000.0,1000.0,21.0,177.0,70.0,Right,2.0,2.0,...,43,43,43,44,47,49,49,49,47,17.0
18940,47.0,53.0,70000.0,1000.0,21.0,174.0,68.0,Right,2.0,2.0,...,50,50,50,49,49,48,48,48,49,17.0
18941,47.0,47.0,45000.0,2000.0,28.0,185.0,79.0,Right,2.0,2.0,...,47,47,47,47,47,47,47,47,47,13.0
18942,47.0,67.0,130000.0,500.0,17.0,171.0,58.0,Right,2.0,2.0,...,34,34,34,37,35,28,28,28,35,16.0


In [240]:
for column in players_21.columns[:69]:
    if players_21[column].dtype == 'object':
        encoder = ce.BinaryEncoder(cols=[column])
        col_encoded = encoder.fit_transform(players_21[column])
        players_21.drop(column, axis=1, inplace=True)
        players_21 = pd.concat([players_21, col_encoded], axis=1).reset_index(drop=True)

In [241]:
correlations = players_21.corr()['overall']
sorted_correlations = correlations.sort_values(ascending=False)

threshold = 0.5
players_21_relevant_columns = sorted_correlations[abs(sorted_correlations) > threshold]


players_21_relevant_columns.index

Index(['overall', 'movement_reactions', 'mentality_composure', 'passing',
       'potential', 'dribbling', 'wage_eur', 'power_shot_power', 'value_eur',
       'mentality_vision', 'attacking_short_passing'],
      dtype='object')

In [242]:
players_21 = players_21[players_21_relevant_columns.index]

In [243]:
players_21

,overall,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,mentality_vision,attacking_short_passing
0,93.0,94.0,96.0,91.0,93.0,95.0,560000.0,86.0,103500000.0,95.0,91.0
1,92.0,95.0,95.0,81.0,92.0,89.0,220000.0,94.0,63000000.0,82.0,82.0
2,91.0,93.0,88.0,78.0,91.0,85.0,240000.0,89.0,111000000.0,79.0,84.0
3,91.0,91.0,93.0,86.0,91.0,94.0,270000.0,80.0,132000000.0,90.0,87.0
4,91.0,91.0,91.0,93.0,91.0,88.0,370000.0,91.0,129000000.0,94.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,48.0,40.0,26.0,52.0,27.0,1000.0,35.0,70000.0,25.0,31.0
18940,47.0,50.0,35.0,49.0,53.0,47.0,1000.0,38.0,70000.0,53.0,55.0
18941,47.0,44.0,35.0,49.0,47.0,46.0,2000.0,49.0,45000.0,44.0,56.0
18942,47.0,53.0,45.0,40.0,67.0,53.0,500.0,50.0,130000.0,49.0,45.0


In [244]:
players_22

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962,https://sofifa.com/player/261962/defu-song/220002,Song Defu,宋德福,CDM,47,52,70000.0,1000.0,22,...,46+2,46+2,46+2,48+2,15+2,https://cdn.sofifa.net/players/261/962/22_120.png,https://cdn.sofifa.net/teams/112541/60.png,https://cdn.sofifa.net/flags/cn.png,NaN,https://cdn.sofifa.net/flags/cn.png
19235,262040,https://sofifa.com/player/262040/caoimhin-port...,C. Porter,Caoimhin Porter,CM,47,59,110000.0,500.0,19,...,44+2,44+2,44+2,48+2,14+2,https://cdn.sofifa.net/players/262/040/22_120.png,https://cdn.sofifa.net/teams/445/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19236,262760,https://sofifa.com/player/262760/nathan-logue/...,N. Logue,Nathan Logue-Cunningham,CM,47,55,100000.0,500.0,21,...,45+2,45+2,45+2,47+2,12+2,https://cdn.sofifa.net/players/262/760/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png
19237,262820,https://sofifa.com/player/262820/luke-rudden/2...,L. Rudden,Luke Rudden,ST,47,60,110000.0,500.0,19,...,26+2,26+2,26+2,32+2,15+2,https://cdn.sofifa.net/players/262/820/22_120.png,https://cdn.sofifa.net/teams/111131/60.png,https://cdn.sofifa.net/flags/ie.png,NaN,https://cdn.sofifa.net/flags/ie.png


In [245]:
players_22.describe()

,sofifa_id,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,19178.000000,...,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,231468.086959,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,50580.498123,1.354364,...,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,27039.717497,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,54401.868535,0.747865,...,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,41.000000,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,...,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,214413.500000,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,...,50.000000,29.000000,28.000000,25.000000,8.000000,8.000000,8.000000,8.000000,8.000000,27.000000
50%,236543.000000,66.000000,71.000000,9.750000e+05,3000.000000,25.000000,181.000000,75.000000,1938.000000,1.000000,...,59.000000,52.000000,56.000000,53.000000,11.000000,11.000000,11.000000,11.000000,11.000000,36.000000
75%,253532.500000,70.000000,75.000000,2.000000e+06,8000.000000,29.000000,186.000000,80.000000,111139.000000,1.000000,...,66.000000,63.000000,65.000000,63.000000,14.000000,14.000000,14.000000,14.000000,14.000000,45.000000
max,264640.000000,93.000000,95.000000,1.940000e+08,350000.000000,54.000000,206.000000,110.000000,115820.000000,5.000000,...,96.000000,93.000000,93.000000,92.000000,91.000000,92.000000,93.000000,92.000000,90.000000,65.000000


In [246]:
players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [247]:
object_columns = players_22.select_dtypes(include='object').columns

print(object_columns)

Index(['player_url', 'short_name', 'long_name', 'player_positions', 'dob',
       'club_name', 'league_name', 'club_position', 'club_loaned_from',
       'club_joined', 'nationality_name', 'nation_position', 'preferred_foot',
       'work_rate', 'body_type', 'real_face', 'player_tags', 'player_traits',
       'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram',
       'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb',
       'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url',
       'club_flag_url', 'nation_logo_url', 'nation_flag_url'],
      dtype='object')


In [248]:
columns_to_drop = players_22.filter(regex='url|dob').columns

players_22.drop(columns=columns_to_drop, axis=1, inplace=True)

In [249]:
players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 103 entries, sofifa_id to gk
dtypes: float64(16), int64(44), object(43)
memory usage: 15.1+ MB


In [250]:
players_22.dropna(thresh= len(players_22) * 0.5, axis=1, inplace=True)

In [251]:
players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  int64  
 1   short_name                   19239 non-null  object 
 2   long_name                    19239 non-null  object 
 3   player_positions             19239 non-null  object 
 4   overall                      19239 non-null  int64  
 5   potential                    19239 non-null  int64  
 6   value_eur                    19165 non-null  float64
 7   wage_eur                     19178 non-null  float64
 8   age                          19239 non-null  int64  
 9   height_cm                    19239 non-null  int64  
 10  weight_kg                    19239 non-null  int64  
 11  club_team_id                 19178 non-null  float64
 12  club_name                    19178 non-null  object 
 13  league_name     

In [252]:
from sklearn.impute import SimpleImputer

numeric_cols = players_22.select_dtypes(include=['number']).columns
num_imputer = SimpleImputer(strategy='median')
players_22[numeric_cols] = num_imputer.fit_transform(players_22[numeric_cols])

obj_imputer = SimpleImputer(strategy='most_frequent')
players_22[players_22.select_dtypes(include='object').columns] = obj_imputer.fit_transform(players_22[players_22.select_dtypes(include='object').columns])

In [253]:
for col in players_22.columns:
    # Convert column to string for checking patterns
    col_str = players_22[col].astype(str)

    # Check if the column contains any data in the form of "number+number" or "number-number"
    if col_str.str.contains(r'^\d+[\+\-]\d+$').any():
        # Apply eval to each element in the column if it matches the pattern
        players_22[col] = players_22[col].apply(lambda x: eval(x) if isinstance(x, str) and re.match(r'^\d+[\+\-]\d+$', x) else x)

        # Convert the column to int or float as needed
        if players_22[col].apply(lambda x: isinstance(x, (int, float))).all():
            players_22[col] = players_22[col].astype(float)

In [254]:
players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 96 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   sofifa_id                    19239 non-null  float64
 1   short_name                   19239 non-null  object 
 2   long_name                    19239 non-null  object 
 3   player_positions             19239 non-null  object 
 4   overall                      19239 non-null  float64
 5   potential                    19239 non-null  float64
 6   value_eur                    19239 non-null  float64
 7   wage_eur                     19239 non-null  float64
 8   age                          19239 non-null  float64
 9   height_cm                    19239 non-null  float64
 10  weight_kg                    19239 non-null  float64
 11  club_team_id                 19239 non-null  float64
 12  club_name                    19239 non-null  object 
 13  league_name     

In [255]:
players_22

,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,158023.0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93.0,93.0,78000000.0,320000.0,34.0,170.0,...,67,67,67,69,64,53,53,53,64,22.0
1,188545.0,R. Lewandowski,Robert Lewandowski,ST,92.0,92.0,119500000.0,270000.0,32.0,185.0,...,69,69,69,67,64,63,63,63,64,22.0
2,20801.0,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91.0,91.0,45000000.0,270000.0,36.0,187.0,...,62,62,62,66,63,56,56,56,63,23.0
3,190871.0,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91.0,91.0,129000000.0,270000.0,29.0,175.0,...,66,66,66,70,65,53,53,53,65,23.0
4,192985.0,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91.0,91.0,125500000.0,350000.0,30.0,181.0,...,83,83,83,82,78,72,72,72,78,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,261962.0,Song Defu,宋德福,CDM,47.0,52.0,70000.0,1000.0,22.0,180.0,...,48,48,48,50,50,48,48,48,50,17.0
19235,262040.0,C. Porter,Caoimhin Porter,CM,47.0,59.0,110000.0,500.0,19.0,175.0,...,48,48,48,51,50,46,46,46,50,16.0
19236,262760.0,N. Logue,Nathan Logue-Cunningham,CM,47.0,55.0,100000.0,500.0,21.0,178.0,...,48,48,48,49,49,47,47,47,49,14.0
19237,262820.0,L. Rudden,Luke Rudden,ST,47.0,60.0,110000.0,500.0,19.0,173.0,...,31,31,31,35,34,28,28,28,34,17.0


In [256]:
for column in players_22.columns[:69]:
    if players_22[column].dtype == 'object':
        encoder = ce.BinaryEncoder(cols=[column])
        col_encoded = encoder.fit_transform(players_22[column])
        players_22.drop(column, axis=1, inplace=True)
        players_22 = pd.concat([players_22, col_encoded], axis=1).reset_index(drop=True)

In [257]:
correlations = players_22.corr()['overall']
sorted_correlations = correlations.sort_values(ascending=False)

threshold = 0.5
players_22_relevant_columns = sorted_correlations[abs(sorted_correlations) > threshold]


players_22_relevant_columns.index

Index(['overall', 'movement_reactions', 'mentality_composure', 'passing',
       'potential', 'dribbling', 'wage_eur', 'power_shot_power', 'value_eur',
       'rcm', 'cm', 'lcm', 'release_clause_eur', 'mentality_vision',
       'attacking_short_passing', 'short_name_0', 'long_name_0'],
      dtype='object')

In [258]:
players_22 = players_22[players_22_relevant_columns.index]

In [259]:
players_22

,overall,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,rcm,cm,lcm,release_clause_eur,mentality_vision,attacking_short_passing,short_name_0,long_name_0
0,93.0,94.0,96.0,91.0,93.0,95.0,320000.0,86.0,78000000.0,90,90,90,144300000.0,95.0,91.0,0,0
1,92.0,93.0,88.0,79.0,92.0,86.0,270000.0,90.0,119500000.0,83,83,83,197200000.0,81.0,85.0,0,0
2,91.0,94.0,95.0,80.0,91.0,88.0,270000.0,94.0,45000000.0,81,81,81,83300000.0,76.0,80.0,0,0
3,91.0,89.0,93.0,86.0,91.0,94.0,270000.0,80.0,129000000.0,85,85,85,238700000.0,90.0,86.0,0,0
4,91.0,91.0,89.0,93.0,91.0,88.0,350000.0,91.0,125500000.0,91,91,91,232200000.0,94.0,94.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,47.0,53.0,37.0,46.0,52.0,48.0,1000.0,46.0,70000.0,48,48,48,114000.0,43.0,50.0,1,1
19235,47.0,49.0,47.0,50.0,59.0,46.0,500.0,49.0,110000.0,49,49,49,193000.0,49.0,51.0,0,1
19236,47.0,46.0,36.0,45.0,55.0,49.0,500.0,50.0,100000.0,49,49,49,175000.0,46.0,49.0,1,1
19237,47.0,48.0,47.0,36.0,60.0,48.0,500.0,48.0,110000.0,41,41,41,239000.0,40.0,38.0,1,1


In [260]:
columns_to_remove = [col for col in players_21.columns if col not in players_22.columns]
if columns_to_remove:
    players_21.drop(columns=columns_to_remove, inplace=True)

In [261]:
columns_to_remove = [col for col in players_22.columns if col not in players_21.columns]
if columns_to_remove:
    players_22.drop(columns=columns_to_remove, inplace=True)

In [262]:
players_21

,overall,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,mentality_vision,attacking_short_passing
0,93.0,94.0,96.0,91.0,93.0,95.0,560000.0,86.0,103500000.0,95.0,91.0
1,92.0,95.0,95.0,81.0,92.0,89.0,220000.0,94.0,63000000.0,82.0,82.0
2,91.0,93.0,88.0,78.0,91.0,85.0,240000.0,89.0,111000000.0,79.0,84.0
3,91.0,91.0,93.0,86.0,91.0,94.0,270000.0,80.0,132000000.0,90.0,87.0
4,91.0,91.0,91.0,93.0,91.0,88.0,370000.0,91.0,129000000.0,94.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...
18939,47.0,48.0,40.0,26.0,52.0,27.0,1000.0,35.0,70000.0,25.0,31.0
18940,47.0,50.0,35.0,49.0,53.0,47.0,1000.0,38.0,70000.0,53.0,55.0
18941,47.0,44.0,35.0,49.0,47.0,46.0,2000.0,49.0,45000.0,44.0,56.0
18942,47.0,53.0,45.0,40.0,67.0,53.0,500.0,50.0,130000.0,49.0,45.0


In [263]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  18944 non-null  float64
 1   movement_reactions       18944 non-null  float64
 2   mentality_composure      18944 non-null  float64
 3   passing                  18944 non-null  float64
 4   potential                18944 non-null  float64
 5   dribbling                18944 non-null  float64
 6   wage_eur                 18944 non-null  float64
 7   power_shot_power         18944 non-null  float64
 8   value_eur                18944 non-null  float64
 9   mentality_vision         18944 non-null  float64
 10  attacking_short_passing  18944 non-null  float64
dtypes: float64(11)
memory usage: 1.6 MB


In [264]:
players_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  float64
 1   movement_reactions       19239 non-null  float64
 2   mentality_composure      19239 non-null  float64
 3   passing                  19239 non-null  float64
 4   potential                19239 non-null  float64
 5   dribbling                19239 non-null  float64
 6   wage_eur                 19239 non-null  float64
 7   power_shot_power         19239 non-null  float64
 8   value_eur                19239 non-null  float64
 9   mentality_vision         19239 non-null  float64
 10  attacking_short_passing  19239 non-null  float64
dtypes: float64(11)
memory usage: 1.6 MB


In [265]:
y_train = players_21['overall']
X_train = players_21.drop('overall', axis=1)


y_test = players_22['overall']
X_test = players_22.drop('overall', axis=1)

In [266]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [267]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [268]:
X_train.shape

(15155, 10)

In [269]:
X_test.shape

(3789, 10)

In [270]:
X_train.head()

,movement_reactions,mentality_composure,passing,potential,dribbling,wage_eur,power_shot_power,value_eur,mentality_vision,attacking_short_passing
18775,32.0,21.0,58.0,64.0,64.0,500.0,35.0,100000.0,34.0,15.0
17559,51.0,52.0,52.0,74.0,58.0,750.0,41.0,325000.0,56.0,60.0
7667,64.0,74.0,66.0,67.0,68.0,7000.0,71.0,1000000.0,66.0,70.0
2638,73.0,66.0,62.0,73.0,67.0,5000.0,65.0,2600000.0,51.0,66.0
18148,50.0,38.0,35.0,70.0,50.0,1000.0,26.0,230000.0,36.0,33.0


In [271]:
y_train.head()

18775    50.0
17559    55.0
7667     67.0
2638     73.0
18148    53.0
Name: overall, dtype: float64

In [272]:
# Multi Linear Regression
from sklearn.linear_model import LinearRegression

l = LinearRegression()

In [273]:
l.fit(X_train,y_train)

LinearRegression()

In [274]:
y_pred = l.predict(X_test)

In [275]:
X_train.columns

Index(['movement_reactions', 'mentality_composure', 'passing', 'potential',
       'dribbling', 'wage_eur', 'power_shot_power', 'value_eur',
       'mentality_vision', 'attacking_short_passing'],
      dtype='object')

In [276]:
intercept = l.intercept_

coefficients = l.coef_

print(f"Intercept: {intercept}")
print(f"Coefficients: {coefficients}")

Intercept: 12.18626317635458
Coefficients: [ 3.98773208e-01  1.24926264e-01  1.77295873e-01  2.17383246e-01
  1.50367741e-02  2.95003183e-05  3.17512301e-02  1.01844654e-08
 -1.08163145e-01 -2.00265850e-02]


In [277]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [278]:
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.0793346402707384,
Mean Squared Error = 7.150381288414989,
Root Mean Squared Error = 2.6740196873648836,
R2 Score = 0.8277122643706206



In [279]:
# Polynomial Regression

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

poly_pipeline = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())

poly = PolynomialFeatures(degree=1)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.fit_transform(X_test)


model = LinearRegression()
model.fit(X_poly_train, y_train)


y_pred = model.predict(X_poly_test)

In [280]:
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred, y_test)},
Mean Squared Error = {mean_squared_error(y_pred, y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, y_test))},
R2 Score = {r2_score(y_pred, y_test)}
""")


Mean Absolute Error = 2.0793346402648187,
Mean Squared Error = 7.150381288403349,
Root Mean Squared Error = 2.6740196873627067,
R2 Score = 0.8277122643654089



In [281]:
poly = PolynomialFeatures(degree=2)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.fit_transform(X_test)

model = LinearRegression()
model.fit(X_poly_train, y_train)

y_pred = model.predict(X_poly_test)
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred, y_test)},
Mean Squared Error = {mean_squared_error(y_pred, y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, y_test))},
R2 Score = {r2_score(y_pred, y_test)}
""")


Mean Absolute Error = 1.6677596592973885,
Mean Squared Error = 4.851072388091284,
Root Mean Squared Error = 2.202515014271477,
R2 Score = 0.8875824702950315



In [282]:
poly = PolynomialFeatures(degree=3)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.fit_transform(X_test)

model = LinearRegression()
model.fit(X_poly_train, y_train)

y_pred = model.predict(X_poly_test)
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred, y_test)},
Mean Squared Error = {mean_squared_error(y_pred, y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, y_test))},
R2 Score = {r2_score(y_pred, y_test)}
""")


Mean Absolute Error = 5.992396814264434,
Mean Squared Error = 360.98840809326964,
Root Mean Squared Error = 18.999694947374014,
R2 Score = 0.04069006019949495



In [283]:
# Regularisation Models
from sklearn.linear_model import ElasticNet, Lasso, Ridge


model1 = Ridge()
model1.fit(X_train, y_train)


y_pred = model1.predict(X_test)

In [284]:
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.07933449674686,
Mean Squared Error = 7.15038140143443,
Root Mean Squared Error = 2.6740197084977573,
R2 Score = 0.8277121283746938



In [285]:
model2 = Lasso()
model2.fit(X_train, y_train)


y_pred = model2.predict(X_test)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.0891808196171135,
Mean Squared Error = 7.2941080742387046,
Root Mean Squared Error = 2.7007606473433934,
R2 Score = 0.8161825869148481



In [286]:
model3 = ElasticNet()
model3.fit(X_train, y_train)


y_pred = model3.predict(X_test)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 2.0788078668034884,
Mean Squared Error = 7.19745424183022,
Root Mean Squared Error = 2.6828071570335092,
R2 Score = 0.8215580254594149



In [287]:
from sklearn.tree import DecisionTreeRegressor
dtree = DecisionTreeRegressor(max_depth=2000)

In [288]:
dtree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2000)

In [289]:
y_pred = dtree.predict(X_test)
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 0.8701504354711006,
Mean Squared Error = 3.02164159408815,
Root Mean Squared Error = 1.7382869711552664,
R2 Score = 0.9357177722516008



In [290]:
from sklearn.ensemble import StackingRegressor


estimators = [
    ('lr', l),
    ('poly', poly_pipeline),
    ('rm1', model1),
    ('rm2', model2),
    ('rm3', model3),
    ('dt', dtree)
]


stacking_model = StackingRegressor(estimators=estimators, final_estimator=DecisionTreeRegressor
(max_depth=10000000000000000))


stacking_model.fit(X_train, y_train)


y_pred = stacking_model.predict(X_test)


print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,y_test)},
Mean Squared Error = {mean_squared_error(y_pred,y_test)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,y_test))},
R2 Score = {r2_score(y_pred,y_test)}
""")


Mean Absolute Error = 1.4135655845869624,
Mean Squared Error = 5.09633148588018,
Root Mean Squared Error = 2.257505589335313,
R2 Score = 0.8939749974352199



In [291]:
from sklearn.model_selection import cross_val_score, GridSearchCV


cv_scores = cross_val_score(stacking_model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')


mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)

print(f"Mean CV Score (Negative MSE): {mean_cv_score}")
print(f"Standard Deviation of CV Scores: {std_cv_score}")

Mean CV Score (Negative MSE): -3.8081821181128332
Standard Deviation of CV Scores: 0.4274440434980664


In [292]:
param_grid = {
    'final_estimator__max_depth': [5, 10, 15, 20],
    'final_estimator__min_samples_split': [2, 5, 10]
}


grid_search = GridSearchCV(stacking_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)


grid_search.fit(X_train_scaled, y_train)


best_model = grid_search.best_estimator_


y_pred_best = best_model.predict(X_test_scaled)


print(f"Best Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_best)}")
print(f"Best Model Mean Squared Error: {mean_squared_error(y_test, y_pred_best)}")
print(f"Best Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_best))}")
print(f"Best Model R2 Score: {r2_score(y_test, y_pred_best)}")


Best Model Mean Absolute Error: 1.0140627010911998
Best Model Mean Squared Error: 2.0935318118601685
Best Model Root Mean Squared Error: 1.446904216546544
Best Model R2 Score: 0.9555843224635037


In [293]:
refined_param_grid = {
    'final_estimator__max_depth': [12, 15, 18],
    'final_estimator__min_samples_split': [5, 7, 10]
}


refined_grid_search = GridSearchCV(stacking_model, refined_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)


refined_grid_search.fit(X_train_scaled, y_train)


best_refined_model = refined_grid_search.best_estimator_

In [294]:
best_refined_model.fit(X_train_scaled, y_train)

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('poly',
                               Pipeline(steps=[('polynomialfeatures',
                                                PolynomialFeatures(degree=3)),
                                               ('linearregression',
                                                LinearRegression())])),
                              ('rm1', Ridge()), ('rm2', Lasso()),
                              ('rm3', ElasticNet()),
                              ('dt', DecisionTreeRegressor(max_depth=2000))],
                  final_estimator=DecisionTreeRegressor(max_depth=12,
                                                        min_samples_split=10))

In [295]:
y_pred_final = best_refined_model.predict(X_test_scaled)

print(f"Redefined Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_final)}")
print(f"Redefined Model Mean Squared Error: {mean_squared_error(y_test, y_pred_final)}")
print(f"Redefined Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_final))}")
print(f"Redefined Model R2 Score: {r2_score(y_test, y_pred_final)}")

Redefined Model Mean Absolute Error: 0.9750112404574979
Redefined Model Mean Squared Error: 2.3282396802602574
Redefined Model Root Mean Squared Error: 1.5258570313958832
Redefined Model R2 Score: 0.9506048380634676


In [296]:
refined_param_grid = {
    'final_estimator__max_depth': [10, 15],
    'final_estimator__min_samples_split': [5, 10]
}


refined_grid_search = GridSearchCV(stacking_model, refined_param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)


refined_grid_search.fit(X_train_scaled, y_train)


best_refined_model = refined_grid_search.best_estimator_

In [297]:
best_refined_model.fit(X_train_scaled, y_train)

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('poly',
                               Pipeline(steps=[('polynomialfeatures',
                                                PolynomialFeatures(degree=3)),
                                               ('linearregression',
                                                LinearRegression())])),
                              ('rm1', Ridge()), ('rm2', Lasso()),
                              ('rm3', ElasticNet()),
                              ('dt', DecisionTreeRegressor(max_depth=2000))],
                  final_estimator=DecisionTreeRegressor(max_depth=10,
                                                        min_samples_split=10))

In [298]:
y_pred_final = best_refined_model.predict(X_test_scaled)

print(f"Final Model Mean Absolute Error: {mean_absolute_error(y_test, y_pred_final)}")
print(f"Final Model Mean Squared Error: {mean_squared_error(y_test, y_pred_final)}")
print(f"Final Model Root Mean Squared Error: {np.sqrt(mean_squared_error(y_test, y_pred_final))}")
print(f"Final Model R2 Score: {r2_score(y_test, y_pred_final)}")

Final Model Mean Absolute Error: 0.9112673516230082
Final Model Mean Squared Error: 1.9608100798821673
Final Model Root Mean Squared Error: 1.4002892843559744
Final Model R2 Score: 0.9584001027713187


In [299]:
joblib.dump(best_refined_model, 'final_stacking_model.pkl')

['final_stacking_model.pkl']